In [15]:

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions as f
from pyspark.sql.types import *
import datetime
import dateutil.relativedelta
import os
import sys
import argparse

spark = SparkSession.builder \
.master("local") \
.appName("CS in HT") \
.getOrCreate()

In [16]:
paid = spark.read.option("header",True).option("inferSchema",True).option("delimiter",";")\
   .csv("file:////home/abhijeet/spark coding/Data_sample/Paid_Services_sample.csv")
paid.printSchema()
paid.registerTempTable('p')
paid.show(2)

root
 |-- Clearing_Period: integer (nullable = true)
 |-- OIB: string (nullable = true)
 |-- Document_Number: long (nullable = true)
 |-- Document_Type: string (nullable = true)
 |-- Days_between_Due_and_Clearing_Date: integer (nullable = true)
 |-- Amount: double (nullable = true)
 |-- IS_Cleared_with_Payment: integer (nullable = true)
 |-- Cleared_with_Document_Number: long (nullable = true)
 |-- Cleared_with_Document_Type: string (nullable = true)

+---------------+-----------+---------------+-------------+----------------------------------+------+-----------------------+----------------------------+--------------------------+
|Clearing_Period|        OIB|Document_Number|Document_Type|Days_between_Due_and_Clearing_Date|Amount|IS_Cleared_with_Payment|Cleared_with_Document_Number|Cleared_with_Document_Type|
+---------------+-----------+---------------+-------------+----------------------------------+------+-----------------------+----------------------------+--------------------------

In [17]:
collvl = spark.read.option("header",True).option("inferSchema",True).option("delimiter",";")\
   .csv("file:////home/abhijeet/spark coding/Data_sample/CollectionLevel_by_DocNo_sample.csv")
collvl.printSchema()
collvl.show(2)
collvl.registerTempTable('c')


root
 |-- Document_Number: long (nullable = true)
 |-- Due_Date: integer (nullable = true)
 |-- Dunning_Date: integer (nullable = true)
 |-- Collection_Level: integer (nullable = true)
 |-- Dunning_Amount: double (nullable = true)

+---------------+--------+------------+----------------+--------------+
|Document_Number|Due_Date|Dunning_Date|Collection_Level|Dunning_Amount|
+---------------+--------+------------+----------------+--------------+
|      210551287|20181016|    20190716|              20|         74.16|
|      210547151|20181217|    20190627|              80|          60.0|
+---------------+--------+------------+----------------+--------------+
only showing top 2 rows



In [22]:
paid_df = spark.sql("select * from p where Document_Type not in ('CF','EN','IC','NP','PO','SR')")
paid_df.show(2)

+---------------+-----------+---------------+-------------+----------------------------------+------+-----------------------+----------------------------+--------------------------+
|Clearing_Period|        OIB|Document_Number|Document_Type|Days_between_Due_and_Clearing_Date|Amount|IS_Cleared_with_Payment|Cleared_with_Document_Number|Cleared_with_Document_Type|
+---------------+-----------+---------------+-------------+----------------------------------+------+-----------------------+----------------------------+--------------------------+
|         201707|50270059011|    20331874213|           RK|                                -3| 366.5|                      1|                 44624209266|                        UZ|
|         201708|23386350591|    20179457076|           RB|                                 3| 80.08|                      1|                 44823969505|                        UN|
+---------------+-----------+---------------+-------------+-------------------------------

+---------------+---------------+---+-------------+----------------------------------+------+-----------------------+----------------------------+--------------------------+--------+------------+----------------+--------------+
|Document_Number|Clearing_Period|OIB|Document_Type|Days_between_Due_and_Clearing_Date|Amount|IS_Cleared_with_Payment|Cleared_with_Document_Number|Cleared_with_Document_Type|Due_Date|Dunning_Date|Collection_Level|Dunning_Amount|
+---------------+---------------+---+-------------+----------------------------------+------+-----------------------+----------------------------+--------------------------+--------+------------+----------------+--------------+
+---------------+---------------+---+-------------+----------------------------------+------+-----------------------+----------------------------+--------------------------+--------+------------+----------------+--------------+



In [25]:
#df1 = paid_df.join(collvl,"Document_Number","inner")
#df1.show()
joindata = spark.read.option("header",True).option("inferSchema",True).option("delimiter",";")\
   .csv("file:////home/abhijeet/spark coding/paid.csv")
joindata.printSchema()
joindata.registerTempTable('pd')
joindata.show(2)

root
 |-- document_number: long (nullable = true)
 |-- clearing_period: integer (nullable = true)
 |-- oib: long (nullable = true)
 |-- document_type: string (nullable = true)
 |-- days_between_due_and_clearing_date: integer (nullable = true)
 |-- amount: double (nullable = true)
 |-- is_cleared_with_payment: integer (nullable = true)
 |-- cleared_with_document_number: long (nullable = true)
 |-- cleared_with_document_type: string (nullable = true)
 |-- due_date: integer (nullable = true)
 |-- dunning_date: integer (nullable = true)
 |-- collection_level: integer (nullable = true)
 |-- dunning_amount: double (nullable = true)

+---------------+---------------+-----------+-------------+----------------------------------+------+-----------------------+----------------------------+--------------------------+--------+------------+----------------+--------------+
|document_number|clearing_period|        oib|document_type|days_between_due_and_clearing_date|amount|is_cleared_with_payment|clea

In [66]:
df2 = joindata.withColumn("intrvl",f.expr("case when collection_level>=10 and collection_level<=19 then 1019\
                                        when collection_level>=20 and collection_level<=29 then 2029\
                                        when collection_level>=30 and collection_level<=39 then 3039\
                                        when collection_level>=40 and collection_level<=49 then 4049\
                                        when collection_level>=50 and collection_level<=79 then 5079\
                                        when collection_level>=80 and collection_level<=94 then 8094\
                                        when collection_level=95 then 95\
                                        when collection_level=0 then 'No_DUN' end"))
df2.show()
df2.registerTempTable('pd')

+---------------+---------------+-----------+-------------+----------------------------------+------+-----------------------+----------------------------+--------------------------+--------+------------+----------------+--------------+------+
|document_number|clearing_period|        oib|document_type|days_between_due_and_clearing_date|amount|is_cleared_with_payment|cleared_with_document_number|cleared_with_document_type|due_date|dunning_date|collection_level|dunning_amount|intrvl|
+---------------+---------------+-----------+-------------+----------------------------------+------+-----------------------+----------------------------+--------------------------+--------+------------+----------------+--------------+------+
|    19152608709|         201802|64213507239|           IN|                               309|218.25|                      0|                 11030048680|                        OR|20170418|    20180130|              80|        436.67|  8094|
|    19152608709|         20

In [82]:
query1 = spark.sql("""SELECT document_number,clearing_period,dunning_date, oib,intrvl, amount 
from pd """).show()

+---------------+---------------+------------+-----------+------+------+
|document_number|clearing_period|dunning_date|        oib|intrvl|amount|
+---------------+---------------+------------+-----------+------+------+
|    19152608709|         201802|    20180130|64213507239|  8094|218.25|
|    19152608709|         201802|    20180220|64213507239|  8094|218.25|
|    19152608709|         201802|    20180105|64213507239|  5079|218.25|
|    19152608709|         201802|    20180130|64213507239|  8094|218.42|
|    19152608709|         201802|    20180220|64213507239|  8094|218.42|
|    19152608709|         201802|    20180105|64213507239|  5079|218.42|
|    19152615136|         201711|    20171117| 8091698648|  8094|154.17|
|    19152615136|         201711|    20171117| 8091698648|  8094|154.05|
|    19152652432|         201711|    20171117|42170157763|  8094|159.06|
|    19152652432|         201711|    20171117|42170157763|  8094|158.94|
|    19152691929|         201802|    20180130|50475

In [83]:
query1 = spark.sql("""SELECT document_number,clearing_period,\
substr(dunning_date,0,6) as dunning_date,oib,intrvl,amount from pd """)
query1.registerTempTable('f')
query1.show()

+---------------+---------------+------------+-----------+------+------+
|document_number|clearing_period|dunning_date|        oib|intrvl|amount|
+---------------+---------------+------------+-----------+------+------+
|    19152608709|         201802|      201801|64213507239|  8094|218.25|
|    19152608709|         201802|      201802|64213507239|  8094|218.25|
|    19152608709|         201802|      201801|64213507239|  5079|218.25|
|    19152608709|         201802|      201801|64213507239|  8094|218.42|
|    19152608709|         201802|      201802|64213507239|  8094|218.42|
|    19152608709|         201802|      201801|64213507239|  5079|218.42|
|    19152615136|         201711|      201711| 8091698648|  8094|154.17|
|    19152615136|         201711|      201711| 8091698648|  8094|154.05|
|    19152652432|         201711|      201711|42170157763|  8094|159.06|
|    19152652432|         201711|      201711|42170157763|  8094|158.94|
|    19152691929|         201802|      201801|50475

In [84]:
query2 = spark.sql("""SELECT document_number,clearing_period,\
dunning_date,oib,intrvl,amount from f where dunning_date <= clearing_period""")
query2.registerTempTable('fi')
query2.show(2)

+---------------+---------------+------------+-----------+------+------+
|document_number|clearing_period|dunning_date|        oib|intrvl|amount|
+---------------+---------------+------------+-----------+------+------+
|    19152608709|         201802|      201801|64213507239|  8094|218.25|
|    19152608709|         201802|      201802|64213507239|  8094|218.25|
+---------------+---------------+------------+-----------+------+------+
only showing top 2 rows



In [81]:
query3 = spark.sql("""SELECT document_number,clearing_period,\
max(dunning_date) as dunning_date,oib,intrvl,amount from fi \
group by document_number,clearing_period,dunning_date,oib,intrvl,amount """)
query3.show()
query3.registerTempTable('fin')

+---------------+---------------+------------+-----------+------+------+
|document_number|clearing_period|dunning_date|        oib|intrvl|amount|
+---------------+---------------+------------+-----------+------+------+
|    19152817026|         201801|      201711|82552742876|  4049| 70.48|
|    19152817026|         201801|      201711|82552742876|  3039| 70.48|
|    19152615136|         201711|      201711| 8091698648|  8094|154.05|
|    19152821906|         201805|      201712|  984288482|  5079|141.97|
|    19152715696|         201802|      201801|48587808395|  5079| 87.94|
|    19152814118|         201806|      201711| 2708352436|  1019|  0.75|
|    19152814118|         201806|      201805| 2708352436|  8094|  0.75|
|    19152821906|         201805|      201801|  984288482|  5079|141.97|
|    19152725001|         201801|      201801|83818796235|  8094|  28.1|
|    19152711303|         201711|      201711|26877778372|  8094| 69.53|
|    19152814118|         201806|      201805| 2708

In [87]:
final = spark.sql("""SELECT clearing_period, oib,intrvl, sum(amount) as Amount
from fin group by clearing_period,oib,intrvl 
order by clearing_period,oib,intrvl """).show(100)

+---------------+-----------+------+------------------+
|clearing_period|        oib|intrvl|            Amount|
+---------------+-----------+------+------------------+
|         201711| 8091698648|  8094|            308.22|
|         201711|25389530714|  3039|             53.29|
|         201711|26877778372|  8094|             139.0|
|         201711|32458645572|  4049|            208.28|
|         201711|42170157763|  8094|             318.0|
|         201711|50088471722|  5079|             259.0|
|         201711|62440478247|  8094|            166.65|
|         201801|82552742876|  3039|             70.48|
|         201801|82552742876|  4049|             70.48|
|         201801|82552742876|  5079|             70.48|
|         201801|83818796235|  5079|             56.18|
|         201801|83818796235|  8094|            112.36|
|         201802|48587808395|  5079|            175.81|
|         201802|48587808395|  8094|            351.62|
|         201802|50475182207|  5079|            